### SASA trajectory analysis.
Works well for protein and ligand residues of up to 10,000 atoms. You will need a pdb file and a trajectory file. Protein and ligand must have DIFFERENT CHAIN IDs. You also need to have naccess and MDAnalysis.
Written by Peter Obi 11/17/2022

In [191]:
import MDAnalysis
from biopandas.pdb import PandasPdb
import pandas as pd
import numpy as np

In [49]:
import glob

In [6]:
topology = 'step5_input.pdb' #change pdb name as needed
trajectory = '5.trr' #change trajecory name as needed
u = MDAnalysis.Universe(topology,trajectory)
protein_ligand = u.select_atoms("protein or (resname P1N)") #change P1N to ligand name
ligand = u.select_atoms("resname P1N")
len(u.trajectory)

6

In [ ]:
for ts in u.trajectory:
    with MDAnalysis.Writer("{}_complex.pdb".format(ts.frame), protein_ligand.n_atoms) as W:
        W.write(protein_ligand)
    with MDAnalysis.Writer("{}_ligand.pdb".format(ts.frame), ligand.n_atoms) as L:
        L.write(ligand)     

In [210]:

ppdb = PandasPdb().read_pdb('0_complex.pdb')
ppdb.df['ATOM']

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,MET,,X,9,,...,63.18,71.94,-10.03,1.0,0.0,,PROA,N,NaN,3
1,ATOM,2,,HT1,,MET,,X,9,,...,62.75,72.45,-10.83,1.0,0.0,,PROA,H,NaN,4
2,ATOM,3,,HT2,,MET,,X,9,,...,64.13,71.71,-10.36,1.0,0.0,,PROA,H,NaN,5
3,ATOM,4,,HT3,,MET,,X,9,,...,63.28,72.71,-9.34,1.0,0.0,,PROA,H,NaN,6
4,ATOM,5,,CA,,MET,,X,9,,...,62.27,70.89,-9.56,1.0,0.0,,PROA,C,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26962,ATOM,26963,,H192,,P1N,,L,509,,...,39.58,26.48,72.49,1.0,0.0,,HETA,H,NaN,26965
26963,ATOM,26964,,H193,,P1N,,L,509,,...,40.58,26.57,73.96,1.0,0.0,,HETA,H,NaN,26966
26964,ATOM,26965,,H211,,P1N,,L,509,,...,38.40,25.53,82.31,1.0,0.0,,HETA,H,NaN,26967
26965,ATOM,26966,,H212,,P1N,,L,509,,...,38.99,24.19,81.40,1.0,0.0,,HETA,H,NaN,26968


In [213]:
ppdb.df.keys()

dict_keys(['ATOM', 'HETATM', 'ANISOU', 'OTHERS'])

In [ ]:
for value in ppdb.df.items():
    if ppdb.df['ATOM']['segment_id']=='PROA':
        ppdb.df['ATOM']['chain_id']='A'
        

In [ ]:
ppdb.df['ATOM']['chain_id'].apply(lambda row: 'A' if row.segment_I > 5 and row.TotalMntProducts > 2000 else 0, axis=1)


In [171]:
df1 = ppdb.df['ATOM'].loc[ppdb.df['ATOM']['segment_id'] == 'HETA']

In [172]:
df1['chain_id'] = 'H'

In [173]:
df = ppdb.df['ATOM'].loc[ppdb.df['ATOM']['segment_id'] == 'PROA']
pd.options.mode.chained_assignment = None

In [174]:
df['chain_id'] = 'P'

In [ ]:
df1

In [207]:
ppdb = PandasPdb().read_pdb_from_list(values)
ppdb.df['ATOM']

/home/guests/petero/.conda/envs/my_jupyter_notebooks/lib/python3.9/site-packages/biopandas/pdb/pandas_pdb.py:461: UserWarning: No ATOM/HETATM entries have been loaded. Is the input file/text in the pdb format?
  warnings.warn(


,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx


In [204]:
values = ','.join(str(v) for v in final)

In [194]:
final = result.to_numpy()

In [195]:
final

array([['ATOM', 1, '', ..., 'N', nan, 3],
       ['ATOM', 2, '', ..., 'H', nan, 4],
       ['ATOM', 3, '', ..., 'H', nan, 5],
       ...,
       ['ATOM', 26965, '', ..., 'H', nan, 26967],
       ['ATOM', 26966, '', ..., 'H', nan, 26968],
       ['ATOM', 26967, '', ..., 'H', nan, 26969]], dtype=object)

In [198]:
final = np.where(pd.isna(final), '', final)

In [202]:
final = final.tolist()

In [182]:
frames = [df, df1]
result = pd.concat(frames)

In [ ]:
pd.options.mode.chained_assignment = None
for filepath in glob.glob(os.path.join('/home/projects/petero', '*.pdb')):
    print(filepath)
    ppdb = PandasPdb().read_pdb(filepath)
    df = ppdb.df['ATOM'].loc[ppdb.df['ATOM']['segment_id'] == 'PROA']
    df1 = ppdb.df['ATOM'].loc[ppdb.df['ATOM']['segment_id'] == 'HETA']
    df['chain_id'] = 'P'
    df1['chain_id'] = 'H'
    frames = [df, df1]
    result = pd.concat(frames)
    ppdb.df['ATOM'] = esult
    ppdb.to_pdb(path='{}'.format(filepath), 
            records=None, 
            gz=False, 
            append_newline=True)

In [ ]:
cmd = '''
cd /home/projects/petero/binding-pocket-project/naccess/test
for filename in /home/projects/petero/binding-pocket-project/naccess/test/*.pdb
do
  /home/projects/petero/software/naccess/naccess "$filename";
done
touch SASA_ligand.txt
touch SASA_complex.txt
for filename in test/*
do
  prefix="${filename##*/}"
  base="${prefix%%_*}"
  echo "$base"_ligand_sasa>>SASA_ligand.txt
  grep "CHAIN  1 H" ${base}_ligand.rsa >>SASA_ligand.txt
  echo "$base"_complex_sasa >>SASA_complex.txt
  grep "CHAIN  2 H" ${base}_complex.rsa >>SASA_complex.txt
done 
'''

In [ ]:
subprocess.run(cmd, shell=True, check=True, executable='/usr/bin/bash')

In [ ]:
df = pd.read_fwf('/home/projects/petero/binding-pocket-project/naccess/test/SASA_ligand.txt', skiprows=1, header = None)
df1 = pd.read_fwf('/home/projects/petero/binding-pocket-project/naccess/test/SASA_complex.txt', skiprows=1, header = None

In [ ]:
df

In [ ]:
df1

In [ ]:
df.columns= ['PDB ID', 'All-atoms', 'Total-Side', 'Main-Chain', 'Non-polar', 'All polar']
df1.columns= ['PDB ID','All-atoms', 'Total-Side', 'Main-Chain', 'Non-polar', 'All polar']

In [ ]:
df

In [ ]:
df1

In [ ]:
df2 = (pd.merge(df, df1, on='PDB ID'))

In [ ]:
df2

In [ ]:
df2['BSA_total'] = df2['All-atoms_x']- df2['All-atoms_y']
df2['MESA_total'] = df2['All-atoms_y']

In [ ]:
df2['BSA_nonpolar'] = df2['Non-polar_x']- df2['Non-polar_y']
df2['MESA_nonpolar'] = df2['Non-polar_y']

In [ ]:
df2['BSA_polar'] = df2['All polar_x']- df2['All polar_y']
df2['MESA_polar'] = df2['All polar_y']

In [ ]:
df2

In [ ]:
df2.to_csv('/home/projects/petero/binding-pocket-project/naccess/test/final_sasa.csv', index=False)